In [55]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


df = pd.read_csv("./CSV/BTCUSDT/BTC_USDT_Last_30_Days_Candlestik_15m.csv")
# df = pd.read_csv("./CSV/EUR/EURUSD_Candlestick_1_Hour_BID_04.05.2003-15.04.2023.csv")
df=df[df['volume']!=0]

df.reset_index(drop=True, inplace=True)

df['EMA'] = ta.ema(df.close, length=150)
df.tail()

df=df[0:800]

In [56]:
EMAsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.open[i], df.close[i])>=df.EMA[i]:
            dnt=0
        if min(df.open[i], df.close[i])<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        EMAsignal[row]=3
    elif upt==1:
        EMAsignal[row]=2
    elif dnt==1:
        EMAsignal[row]=1
df['EMASignal'] = EMAsignal

In [57]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].low > df.iloc[i].low:
            pivotLow=0
        if df.iloc[candle].high < df.iloc[i].high:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [58]:
window=20
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [59]:
def pointpos(x):
    if x['isPivot']==2:
        return x['low']-1e-3
    elif x['isPivot']==1:
        return x['high']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

# Извлечение pivot points
pivot_points = df[df['isPivot'] > 0].copy()
pivot_prices = pivot_points['pointpos'].dropna().values.reshape(-1, 1)

In [60]:
# Кластеризация pivot points
kmeans = KMeans(n_clusters=10, random_state=0).fit(pivot_prices)
pivot_points.loc[pivot_points['pointpos'].notna(), 'cluster'] = kmeans.labels_

# Определение уровней поддержки и сопротивления
resistance_support_levels = kmeans.cluster_centers_

# Визуализация уровней поддержки и сопротивления
dfpl = df[0:800]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=pivot_points.index, y=pivot_points['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")

for level in resistance_support_levels:
    fig.add_hline(y=level[0], line=dict(color='Red', width=1))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

ValueError: n_samples=7 should be >= n_clusters=10.

In [ ]:
# Функция для анализа локальных уровней
def analyze_local_levels(df, window_size, n_clusters):
    local_levels = []
    for start in range(0, len(df) - window_size, window_size):
        end = start + window_size
        local_pivot_points = df.iloc[start:end]
        local_pivot_prices = local_pivot_points['pointpos'].dropna().values.reshape(-1, 1)
        
        if len(local_pivot_prices) > 0:
            kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(local_pivot_prices)
            local_levels.append(kmeans.cluster_centers_)
    
    return np.array(local_levels).reshape(-1, 1)

# Анализ локальных уровней
window_size = 100  # Примерное количество точек данных в одном окне
n_clusters = 5  # Количество кластеров для локального анализа
local_levels = analyze_local_levels(pivot_points, window_size, n_clusters)

# Визуализация глобальных и локальных уровней
dfpl = df[0:800]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.add_scatter(x=pivot_points.index, y=pivot_points['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")

for level in resistance_support_levels:
    fig.add_hline(y=level[0], line=dict(color='Red', width=1))

for level in local_levels:
    fig.add_hline(y=level[0], line=dict(color='Blue', width=1, dash='dash'))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()